# Quick intro to Kernel Memory: install, upload a doc, ask a question

This notebook shows the basic usage of Kernel Memory, via the serverless client.

First of all, install the Kernel Memory dependency.

In [1]:
#r "nuget: Microsoft.KernelMemory.Core, 0.29.240219.2"
#r "nuget: Neo4j.Driver, 5.18.1"
#r "../../src/Neo4jMemoryStorage/bin/Debug/net8.0/Neo4j.KernelMemory.MemoryStorage.dll"

using Microsoft.KernelMemory;
using Neo4j.KernelMemory.MemoryStorage;
using Neo4j.Driver;

Installed Packages Microsoft.KernelMemory.Core, 0.29.240219.2 Neo4j.Driver, 5.18.1

For our demo, we use also the "dotenv" nuget, to load our secret credentials from a `.env` file.
Make sure you create your `.env` file, with this content:

> ```
> OPENAI_API_KEY=<your OpenAI API key>
> ```

In [2]:
#r "nuget: dotenv.net, 3.1.3"

dotenv.net.DotEnv.Load();
var env = dotenv.net.DotEnv.Read();

var neo4jConfig = new Neo4jConfig
{
    Uri = env["NEO4J_URI"],
    Username = env["NEO4J_USERNAME"],
    Password = env["NEO4J_PASSWORD"]
};

Console.WriteLine($"Neo4j URI: {neo4jConfig.Uri}");

Installed Packages dotenv.net, 3.1.3

Neo4j URI: neo4j://localhost:7687


Now let's create an instance of Kernel Memory client using the `KernelMemoryBuilder` to configure our solution. 

In this demo we use OpenAI to calculate embeddings and generate text, and the default storage settings, with content and embeddings kept in a volatile memory automatically deleted after the execution.

In [3]:
var memory = new KernelMemoryBuilder()
    .WithOpenAIDefaults(env["OPENAI_API_KEY"])
    .WithNeo4j(neo4jConfig)
    .Build<MemoryServerless>();

Neo4jMemory created for neo4j://localhost:7687
Neo4jMemory created for neo4j://localhost:7687
info: Microsoft.KernelMemory.Handlers.TextExtractionHandler[0]
      Handler 'extract' ready
info: Microsoft.KernelMemory.Handlers.TextPartitioningHandler[0]
      Handler 'partition' ready
info: Microsoft.KernelMemory.Handlers.SummarizationHandler[0]
      Handler 'summarize' ready
info: Microsoft.KernelMemory.Handlers.GenerateEmbeddingsHandler[0]
      Handler 'gen_embeddings' ready, 1 embedding generators
info: Microsoft.KernelMemory.Handlers.SaveRecordsHandler[0]
      Handler save_records ready, 1 vector storages
info: Microsoft.KernelMemory.Handlers.DeleteDocumentHandler[0]
      Handler 'private_delete_document' ready
info: Microsoft.KernelMemory.Handlers.DeleteIndexHandler[0]
      Handler 'private_delete_index' ready
info: Microsoft.KernelMemory.Handlers.DeleteGeneratedFilesHandler[0]
      Handler 'delete_generated_files' ready


At this point we have the memory client ready, we can import data and ask questions!

Let's import a PDF file with some details about Kernel Memory:

In [4]:
await memory.ImportDocumentAsync("sample-KM-Readme.pdf", documentId: "doc001");

Now, let's ask a question about Kernel Memory:

In [5]:
var question = "What's Kernel Memory?";

var answer = await memory.AskAsync(question);

Console.WriteLine($"Question: {question}\n\nAnswer: {answer.Result}");

Question: What's Kernel Memory?

Answer: Kernel Memory is a multi-modal AI service that specializes in efficient indexing of datasets through custom continuous data hybrid pipelines. It supports features such as Retrieval Augmented Generation (RAG), synthetic memory, prompt engineering, and custom semantic memory processing. It includes a GPT Plugin, web clients, a .NET library for embedded applications, and can be deployed as a Docker container. Kernel Memory utilizes advanced embeddings and LLMs (Large Language Models) to enable natural language querying for obtaining answers from indexed data. It supports various data formats including web pages, PDFs, images, Word, PowerPoint, Excel, Markdown, JSON, and more. It also supports different storage engines such as Azure AI Search, Elasticsearch, Postgres, Qdrant, Redis, SQL Server, and more. Kernel Memory offers features like RAG, summarization, security filters, long-running ingestion, custom tokenization, document storage, OCR via Azu

And here show the sources used to generate the answer above, what we call "citations":

In [6]:
Console.WriteLine("Sources:\n");

foreach (var x in answer.RelevantSources)
{
    Console.WriteLine($"  - {x.SourceName}  - {x.Link} [{x.Partitions.First().LastUpdate:D}]");
}

Sources:

  - sample-KM-Readme.pdf  - default// [Monday, March 25, 2024]
